In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

train_identity = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/train_identity.csv')
train_transaction = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/train_transaction.csv')
test_identity = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/test_identity.csv')
test_transaction = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/test_transaction.csv')
test_identity.columns = [col.replace('-','_')for col in test_identity.columns]

train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [2]:
del train_identity,test_identity,train_transaction,test_transaction 

# 전처리 [컬럼drop(null_80%이상)]

In [3]:
dff_column_to_drop = train.columns[(train.isna().sum() / len(train) > 0.2)]

In [4]:
dff_column_to_drop=dff_column_to_drop.drop(['id_30','id_31','id_32','id_33','id_34','DeviceType','DeviceInfo'])

In [5]:
def preprocess(df) :
    df = df.drop(columns=dff_column_to_drop).copy()
    # object 범주형 자료 결측치 - 최빈값으로 채워 넣기
    # df.ProductCD.value_counts().index # ['W', 'C', 'R', 'H', 'S']
    df.ProductCD.fillna(df.ProductCD.mode(), inplace=True)
    from sklearn.preprocessing import LabelEncoder
    lbl_enc = LabelEncoder()
    lbl_enc.fit(df[['ProductCD']])
    df['ProductCD'] = lbl_enc.transform(df[['ProductCD']])
    # df.card4.value_counts().index # ['visa', 'mastercard', 'american express', 'discover']
    df.card4.fillna(df.card4.mode(), inplace=True)
    lbl_enc.fit(df[['card4']])
    df['card4'] = lbl_enc.transform(df[['card4']])
    # df.card6.value_counts().index # ['debit', 'credit', 'debit or credit', 'charge card']
    df.card6.fillna(df.card6.mode(), inplace=True)
    lbl_enc.fit(df[['card6']])
    df['card6'] = lbl_enc.transform(df[['card6']])
    # df.P_emaildomain.value_counts().index    # [ gmail yahoo etc   ]
    df.P_emaildomain.fillna(df.P_emaildomain.mode(), inplace=True)
    lbl_enc.fit(df[['P_emaildomain']])
    df['P_emaildomain'] = lbl_enc.transform(df[['P_emaildomain']])
    # id_30
    df.id_30.fillna(df.id_30.mode(), inplace=True)
    lbl_enc.fit(df[['id_30']])
    df['id_30'] = lbl_enc.transform(df[['id_30']])
    #id_31
    df.id_31.fillna(df.id_31.mode(), inplace=True)
    lbl_enc.fit(df[['id_31']])
    df['id_31'] = lbl_enc.transform(df[['id_31']])
    #id_33
    df.id_33.fillna(df.id_33.mode(), inplace=True)
    lbl_enc.fit(df[['id_33']])
    df['id_33'] = lbl_enc.transform(df[['id_33']])
    #id_34
    df.id_34.fillna(df.id_34.mode(), inplace=True)
    lbl_enc.fit(df[['id_34']])
    df['id_34'] = lbl_enc.transform(df[['id_34']])
    #DeviceType
    df.DeviceType.fillna(df.DeviceType.mode(), inplace=True)
    lbl_enc.fit(df[['DeviceType']])
    df['DeviceType'] = lbl_enc.transform(df[['DeviceType']])
    #DeviceInfo
    df.DeviceInfo.fillna(df.DeviceInfo.mode(), inplace=True)
    lbl_enc.fit(df[['DeviceInfo']])
    df['DeviceInfo'] = lbl_enc.transform(df[['DeviceInfo']])
    

    #NULL값이 있는지 한눈에 보기
    display(np.sum(df.isnull() ).to_frame().transpose() )
    display(f'지금 {np.sum(np.sum(df.isnull()))} 개의 null값이 있습니다.' )


    # continuous 연속형 자료 결측치 추가
    # 방법 1 - 중앙값으로 채워 넣기
    temp_list_1 = df.columns[df.isnull().sum() > 0]  # 결측치가 있는 컬럼을 골라서 리스트에 넣기
    for aa in temp_list_1 :
        df[aa].fillna(df[aa].median(), inplace=True)  # 중앙값으로 채워 넣기
    # df.info(verbose=True, show_counts=True)

    print("Null :", df.isnull().sum().sum() )

    return df

In [6]:
train = preprocess(train)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V319,V320,V321,id_30,id_31,id_32,id_33,id_34,DeviceType,DeviceInfo
0,0,0,0,0,0,0,8933,1565,0,4259,...,12,12,12,0,0,512954,0,0,0,0


'지금 6082443 개의 null값이 있습니다.'

Null : 0


In [7]:
test = preprocess(test)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V319,V320,V321,id_30,id_31,id_32,id_33,id_34,DeviceType,DeviceInfo
0,0,0,0,0,0,8654,3002,0,4547,0,...,3,3,3,0,0,436020,0,0,0,0


'지금 1500255 개의 null값이 있습니다.'

Null : 0


In [8]:
y = train['isFraud']
x = train.drop('isFraud',axis=1)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.2, random_state=0xC0FFEE)  # 42, 0xC0FFEE

In [10]:
#결측치만 채운 데이터 성능 확인
from sklearn.metrics import accuracy_score, precision_score,recall_score,confusion_matrix

def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision  = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}'.format(accuracy,precision,recall))

In [11]:
def get_model_train_eval(model,ftr_train=None,ftr_test=None,tgt_train=None,tgt_test=None):
    model.fit(ftr_train,tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:,1]
    get_clf_eval(tgt_test,pred)

In [12]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False)
get_model_train_eval(lgbm_clf,ftr_train=x_train,ftr_test=x_val,tgt_train=y_train,tgt_test=y_val)

오차 행렬
[[113703    132]
 [  1545   2728]]
정확도:0.9858, 정밀도:0.9538, 재현율:0.6384


# HyperOpt를 이용한 하이퍼 파라미터 최적화

1.xgboost 

In [13]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                           min_child_weight=int(search_space['min_child_weight']),
                           learning_rate=search_space['learning_rate'],
                           colsample_bytree = search_space['colsample_bytree'],
                           eval_metric ='logloss')
    accuracy = cross_val_score(xgb_clf,x_train,y_train,scoring='accuracy',cv=3)
    
    return {'loss':-1*np.mean(accuracy),'status':STATUS_OK}

In [14]:
from hyperopt import hp
xgb_search_space = {
    'max_depth':hp.quniform('max_depth',5,20,1),
    'min_child_weight':hp.quniform('min_child_weight',1,2,1),
    'learning_rate':hp.uniform('learning_rate',0.01,0.2),
    'colsample_bytree':hp.uniform('colsample_bytree',0.5,1)
}

In [15]:
from hyperopt import fmin,tpe,Trials

trial_val = Trials()
best = fmin(
    fn=objective_func,
    space = xgb_search_space,
    algo = tpe.suggest,
    max_evals = 50,
    trials = trial_val, rstate=np.random.default_rng(seed=9)
)
print('best:',best)

 86%|█▋| 43/50 [11:54:26<1:56:18, 996.89s/trial, best loss: -0.9843427209792233]


KeyboardInterrupt: 

2.lightgbm

In [16]:
lgbm_search_space={
    'num_leaves':hp.quniform('num_leaves',32,64,1),
    'max_depth':hp.quniform('max_depth',100,160,1),
    'min_child_samples':hp.quniform('min_child_samples',60,100,1),
    'subsample':hp.uniform('subsample',0.7,1),
    'learning_rate':hp.uniform('learning_rate',0.01,0.2)
    
}

In [17]:
from sklearn.model_selection import cross_val_score
def objective_func(search_space):
    lgbm_clf = LGBMClassifier(n_estimators=100,
                             num_leaves=int(search_space['num_leaves']),
                             max_depth = int(search_space['min_child_samples']),
                             subsample = search_space['subsample'],
                             learning_rate=search_space['learning_rate'])
    accuracy = cross_val_score(lgbm_clf,x_train,y_train,scoring='accuracy',cv=3)
    
    return {'loss':-1*np.mean(accuracy),'status':STATUS_OK}

In [18]:
from hyperopt import fmin,tpe,Trials

trials=Trials()

best_1 = fmin(fn=objective_func,space=lgbm_search_space,algo=tpe.suggest,
           max_evals=50,trials=trials, rstate=np.random.default_rng(seed=30))
print('best:',best_1)

100%|████████| 50/50 [50:14<00:00, 60.28s/trial, best loss: -0.9800796731642188]
best: {'learning_rate': 0.13751546105270504, 'max_depth': 137.0, 'min_child_samples': 83.0, 'num_leaves': 63.0, 'subsample': 0.7745827853394476}


3.catboost

In [19]:
cat_search_space={
        'depth': hp.quniform("depth", 1, 6, 1),
        'border_count': hp.uniform ('border_count', 32, 255),
        'learning_rate': hp.loguniform('learning_rate', -5.0, -2),
        'l2_leaf_reg': hp.uniform('l2_leaf_reg', 3, 8),
}

In [20]:
from catboost import CatBoostClassifier
def objective_func(search_space):
    cat_clf = CatBoostClassifier(n_estimators=100,
                                learning_rate=search_space['learning_rate'],
                                depth=int(search_space['depth']),
                                eval_metric='AUC',
                                l2_leaf_reg=search_space['l2_leaf_reg'],
                                early_stopping_rounds=50,
                                border_count=int(search_space['border_count']),
                                verbose=False
                                        )
    accuracy = cross_val_score(cat_clf,x_train,y_train,scoring='accuracy',cv=3)
    
    return {'loss':-1*np.mean(accuracy),'status':STATUS_OK}
    

In [21]:
trials = Trials()
best_2 = fmin(
        fn=objective_func,
        space=cat_search_space,
        algo=tpe.suggest,
        max_evals=50,
        trials=trials,
        )
print('best:',best_2)

100%|████████| 50/50 [33:59<00:00, 40.78s/trial, best loss: -0.9751985473924587]
best: {'border_count': 138.22291524256815, 'depth': 6.0, 'l2_leaf_reg': 7.018472938320084, 'learning_rate': 0.13410911207644097}


# CV 세트 기반의 스태킹

바꾸기

In [23]:
#lgbm 최적 하이퍼 파라미터 이용
'''best: {'learning_rate': 0.13751546105270504, 'max_depth': 137.0,
       'min_child_samples': 83.0, 'num_leaves': 63.0, 'subsample': 0.7745827853394476}'''


lgbm_wrapper = LGBMClassifier(
    n_estimators = 400,
    learning_rate = 0.13751546105270504,
    max_dept= 137,
    min_child_samples= 83,
    num_leaves = 63,
    subsample = 0.7745827853394476,
)

In [24]:
#catboost 최적 하이퍼 파라미터 이용
'''best: {'border_count': 138.22291524256815, 'depth': 6.0, 
'l2_leaf_reg': 7.018472938320084, 'learning_rate': 0.13410911207644097}
'''

cat_wrapper = CatBoostClassifier(
    n_estimators = 400,
    border_count = 138,
    depth =6,
    l2_leaf_reg = 7.018472938320084,
    learning_rate = 0.13410911207644097
)

In [27]:
x_train = np.array(x_train)
y_train = np.array(y_train)
test = np.array(test)

In [28]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [29]:
cat_train, cat_test = get_stacking_base_datasets(cat_wrapper, x_train, y_train, test, 7)
#xgb_train, xgb_test = get_stacking_base_datasets(xgb_wrapper, x_over, y_over, test, 7)
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_wrapper, x_train, y_train, test,  7)    

CatBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
0:	learn: 0.4983911	total: 64.3ms	remaining: 25.7s
1:	learn: 0.3665972	total: 130ms	remaining: 25.9s
2:	learn: 0.2816775	total: 199ms	remaining: 26.4s
3:	learn: 0.2277514	total: 282ms	remaining: 27.9s
4:	learn: 0.1929766	total: 375ms	remaining: 29.6s
5:	learn: 0.1661143	total: 452ms	remaining: 29.7s
6:	learn: 0.1489877	total: 542ms	remaining: 30.4s
7:	learn: 0.1389444	total: 615ms	remaining: 30.1s
8:	learn: 0.1305849	total: 683ms	remaining: 29.7s
9:	learn: 0.1250546	total: 778ms	remaining: 30.4s
10:	learn: 0.1197816	total: 859ms	remaining: 30.4s
11:	learn: 0.1167505	total: 923ms	remaining: 29.8s
12:	learn: 0.1144523	total: 999ms	remaining: 29.7s
13:	learn: 0.1125556	total: 1.07s	remaining: 29.5s
14:	learn: 0.1105522	total: 1.16s	remaining: 29.7s
15:	learn: 0.1092828	total: 1.23s	remaining: 29.6s
16:	learn: 0.1083649	total: 1.32s	remaining: 29.9s
17:	learn: 0.1072903	total: 1.4s	remaining: 29.8s
18:	learn: 0.1066518	total: 1.47s	remaining: 

162:	learn: 0.0837069	total: 12.3s	remaining: 17.8s
163:	learn: 0.0836281	total: 12.3s	remaining: 17.7s
164:	learn: 0.0835512	total: 12.4s	remaining: 17.7s
165:	learn: 0.0834551	total: 12.5s	remaining: 17.6s
166:	learn: 0.0833839	total: 12.6s	remaining: 17.6s
167:	learn: 0.0833618	total: 12.7s	remaining: 17.5s
168:	learn: 0.0832530	total: 12.8s	remaining: 17.4s
169:	learn: 0.0831955	total: 12.8s	remaining: 17.3s
170:	learn: 0.0831011	total: 12.9s	remaining: 17.3s
171:	learn: 0.0830308	total: 13s	remaining: 17.2s
172:	learn: 0.0829690	total: 13s	remaining: 17.1s
173:	learn: 0.0829070	total: 13.1s	remaining: 17s
174:	learn: 0.0828252	total: 13.2s	remaining: 17s
175:	learn: 0.0828034	total: 13.3s	remaining: 16.9s
176:	learn: 0.0827274	total: 13.4s	remaining: 16.8s
177:	learn: 0.0826660	total: 13.4s	remaining: 16.8s
178:	learn: 0.0825979	total: 13.5s	remaining: 16.7s
179:	learn: 0.0825053	total: 13.6s	remaining: 16.6s
180:	learn: 0.0824485	total: 13.7s	remaining: 16.6s
181:	learn: 0.082377

321:	learn: 0.0760359	total: 24.2s	remaining: 5.87s
322:	learn: 0.0759573	total: 24.3s	remaining: 5.79s
323:	learn: 0.0759274	total: 24.4s	remaining: 5.71s
324:	learn: 0.0758782	total: 24.4s	remaining: 5.64s
325:	learn: 0.0758274	total: 24.5s	remaining: 5.57s
326:	learn: 0.0758011	total: 24.6s	remaining: 5.49s
327:	learn: 0.0758006	total: 24.7s	remaining: 5.42s
328:	learn: 0.0757451	total: 24.8s	remaining: 5.34s
329:	learn: 0.0757088	total: 24.8s	remaining: 5.27s
330:	learn: 0.0756731	total: 24.9s	remaining: 5.19s
331:	learn: 0.0756491	total: 25s	remaining: 5.12s
332:	learn: 0.0756422	total: 25s	remaining: 5.04s
333:	learn: 0.0755878	total: 25.1s	remaining: 4.97s
334:	learn: 0.0755540	total: 25.2s	remaining: 4.89s
335:	learn: 0.0755185	total: 25.3s	remaining: 4.82s
336:	learn: 0.0754792	total: 25.4s	remaining: 4.74s
337:	learn: 0.0754466	total: 25.4s	remaining: 4.67s
338:	learn: 0.0754035	total: 25.5s	remaining: 4.59s
339:	learn: 0.0753927	total: 25.6s	remaining: 4.51s
340:	learn: 0.07

82:	learn: 0.0901763	total: 6.21s	remaining: 23.7s
83:	learn: 0.0900468	total: 6.28s	remaining: 23.6s
84:	learn: 0.0899838	total: 6.34s	remaining: 23.5s
85:	learn: 0.0898625	total: 6.41s	remaining: 23.4s
86:	learn: 0.0896993	total: 6.5s	remaining: 23.4s
87:	learn: 0.0895890	total: 6.57s	remaining: 23.3s
88:	learn: 0.0895276	total: 6.63s	remaining: 23.2s
89:	learn: 0.0894473	total: 6.7s	remaining: 23.1s
90:	learn: 0.0893032	total: 6.77s	remaining: 23s
91:	learn: 0.0892079	total: 6.84s	remaining: 22.9s
92:	learn: 0.0890279	total: 6.92s	remaining: 22.9s
93:	learn: 0.0888750	total: 7s	remaining: 22.8s
94:	learn: 0.0887807	total: 7.09s	remaining: 22.8s
95:	learn: 0.0887087	total: 7.17s	remaining: 22.7s
96:	learn: 0.0886495	total: 7.24s	remaining: 22.6s
97:	learn: 0.0884980	total: 7.33s	remaining: 22.6s
98:	learn: 0.0884327	total: 7.4s	remaining: 22.5s
99:	learn: 0.0883191	total: 7.47s	remaining: 22.4s
100:	learn: 0.0882150	total: 7.54s	remaining: 22.3s
101:	learn: 0.0881741	total: 7.62s	rem

243:	learn: 0.0791839	total: 18s	remaining: 11.5s
244:	learn: 0.0791281	total: 18s	remaining: 11.4s
245:	learn: 0.0790951	total: 18.1s	remaining: 11.3s
246:	learn: 0.0790168	total: 18.2s	remaining: 11.3s
247:	learn: 0.0789712	total: 18.3s	remaining: 11.2s
248:	learn: 0.0788848	total: 18.4s	remaining: 11.1s
249:	learn: 0.0788517	total: 18.4s	remaining: 11.1s
250:	learn: 0.0787838	total: 18.5s	remaining: 11s
251:	learn: 0.0787568	total: 18.6s	remaining: 10.9s
252:	learn: 0.0787021	total: 18.6s	remaining: 10.8s
253:	learn: 0.0786445	total: 18.7s	remaining: 10.8s
254:	learn: 0.0786138	total: 18.8s	remaining: 10.7s
255:	learn: 0.0785472	total: 18.9s	remaining: 10.6s
256:	learn: 0.0784729	total: 18.9s	remaining: 10.5s
257:	learn: 0.0783775	total: 19s	remaining: 10.5s
258:	learn: 0.0783629	total: 19.1s	remaining: 10.4s
259:	learn: 0.0783034	total: 19.2s	remaining: 10.3s
260:	learn: 0.0782610	total: 19.3s	remaining: 10.3s
261:	learn: 0.0782133	total: 19.3s	remaining: 10.2s
262:	learn: 0.078135

...

87:	learn: 0.0900955	total: 10.5s	remaining: 37.2s
88:	learn: 0.0899655	total: 10.6s	remaining: 37s
89:	learn: 0.0898705	total: 10.7s	remaining: 36.9s
90:	learn: 0.0896579	total: 10.8s	remaining: 36.8s
91:	learn: 0.0895301	total: 11s	remaining: 36.7s
92:	learn: 0.0893648	total: 11.1s	remaining: 36.5s
93:	learn: 0.0892809	total: 11.2s	remaining: 36.4s
94:	learn: 0.0891506	total: 11.3s	remaining: 36.3s
95:	learn: 0.0890592	total: 11.4s	remaining: 36.1s
96:	learn: 0.0889224	total: 11.5s	remaining: 36s
97:	learn: 0.0888234	total: 11.6s	remaining: 35.9s
98:	learn: 0.0887669	total: 11.8s	remaining: 35.7s
99:	learn: 0.0886008	total: 11.9s	remaining: 35.6s
100:	learn: 0.0885003	total: 12s	remaining: 35.5s
101:	learn: 0.0883857	total: 12.1s	remaining: 35.5s
102:	learn: 0.0883180	total: 12.2s	remaining: 35.3s
103:	learn: 0.0882442	total: 12.3s	remaining: 35.1s
104:	learn: 0.0880790	total: 12.5s	remaining: 35s
105:	learn: 0.0879613	total: 12.6s	remaining: 35s
106:	learn: 0.0879028	total: 12.7s	re

248:	learn: 0.0791857	total: 30.3s	remaining: 18.4s
249:	learn: 0.0791605	total: 30.4s	remaining: 18.2s
250:	learn: 0.0791521	total: 30.5s	remaining: 18.1s
251:	learn: 0.0791085	total: 30.6s	remaining: 17.9s
252:	learn: 0.0790759	total: 30.6s	remaining: 17.8s
253:	learn: 0.0790270	total: 30.7s	remaining: 17.6s
254:	learn: 0.0789639	total: 30.8s	remaining: 17.5s
255:	learn: 0.0789248	total: 30.9s	remaining: 17.4s
256:	learn: 0.0788693	total: 31s	remaining: 17.2s
257:	learn: 0.0788314	total: 31s	remaining: 17.1s
258:	learn: 0.0787792	total: 31.1s	remaining: 16.9s
259:	learn: 0.0787498	total: 31.2s	remaining: 16.8s
260:	learn: 0.0787055	total: 31.3s	remaining: 16.7s
261:	learn: 0.0786712	total: 31.3s	remaining: 16.5s
262:	learn: 0.0786401	total: 31.4s	remaining: 16.4s
263:	learn: 0.0785708	total: 31.5s	remaining: 16.2s
264:	learn: 0.0785314	total: 31.6s	remaining: 16.1s
265:	learn: 0.0785096	total: 31.7s	remaining: 15.9s
266:	learn: 0.0784687	total: 31.7s	remaining: 15.8s
267:	learn: 0.07

8:	learn: 0.1304736	total: 1.04s	remaining: 45.4s
9:	learn: 0.1253106	total: 1.17s	remaining: 45.5s
10:	learn: 0.1210224	total: 1.29s	remaining: 45.6s
11:	learn: 0.1181682	total: 1.4s	remaining: 45.2s
12:	learn: 0.1158215	total: 1.52s	remaining: 45.2s
13:	learn: 0.1136506	total: 1.64s	remaining: 45.1s
14:	learn: 0.1115796	total: 1.76s	remaining: 45.2s
15:	learn: 0.1103592	total: 1.87s	remaining: 44.8s
16:	learn: 0.1096240	total: 1.97s	remaining: 44.3s
17:	learn: 0.1080373	total: 2.09s	remaining: 44.3s
18:	learn: 0.1073157	total: 2.21s	remaining: 44.3s
19:	learn: 0.1062776	total: 2.35s	remaining: 44.6s
20:	learn: 0.1055753	total: 2.47s	remaining: 44.6s
21:	learn: 0.1045645	total: 2.62s	remaining: 45s
22:	learn: 0.1039337	total: 2.75s	remaining: 45.1s
23:	learn: 0.1035657	total: 2.86s	remaining: 44.8s
24:	learn: 0.1031647	total: 2.99s	remaining: 44.9s
25:	learn: 0.1026798	total: 3.11s	remaining: 44.8s
26:	learn: 0.1022505	total: 3.24s	remaining: 44.7s
27:	learn: 0.1017643	total: 3.37s	re

171:	learn: 0.0831717	total: 19.4s	remaining: 25.7s
172:	learn: 0.0831319	total: 19.5s	remaining: 25.6s
173:	learn: 0.0830509	total: 19.6s	remaining: 25.5s
174:	learn: 0.0829825	total: 19.7s	remaining: 25.4s
175:	learn: 0.0829223	total: 19.9s	remaining: 25.3s
176:	learn: 0.0828234	total: 20s	remaining: 25.1s
177:	learn: 0.0827517	total: 20s	remaining: 25s
178:	learn: 0.0827031	total: 20.1s	remaining: 24.8s
179:	learn: 0.0826303	total: 20.2s	remaining: 24.7s
180:	learn: 0.0825551	total: 20.3s	remaining: 24.6s
181:	learn: 0.0824930	total: 20.4s	remaining: 24.5s
182:	learn: 0.0824530	total: 20.5s	remaining: 24.3s
183:	learn: 0.0824169	total: 20.6s	remaining: 24.2s
184:	learn: 0.0823893	total: 20.7s	remaining: 24.1s
185:	learn: 0.0823064	total: 20.8s	remaining: 23.9s
186:	learn: 0.0822594	total: 20.9s	remaining: 23.8s
187:	learn: 0.0821896	total: 21s	remaining: 23.7s
188:	learn: 0.0821615	total: 21.1s	remaining: 23.5s
189:	learn: 0.0821181	total: 21.2s	remaining: 23.4s
190:	learn: 0.082047

331:	learn: 0.0758005	total: 42.2s	remaining: 8.64s
332:	learn: 0.0757757	total: 42.3s	remaining: 8.52s
333:	learn: 0.0757270	total: 42.5s	remaining: 8.41s
334:	learn: 0.0757037	total: 42.7s	remaining: 8.29s
335:	learn: 0.0756728	total: 42.9s	remaining: 8.17s
336:	learn: 0.0756661	total: 43s	remaining: 8.04s
337:	learn: 0.0756195	total: 43.2s	remaining: 7.92s
338:	learn: 0.0756170	total: 43.4s	remaining: 7.8s
339:	learn: 0.0755809	total: 43.6s	remaining: 7.69s
340:	learn: 0.0755476	total: 43.7s	remaining: 7.57s
341:	learn: 0.0755063	total: 44s	remaining: 7.46s
342:	learn: 0.0754958	total: 44.1s	remaining: 7.33s
343:	learn: 0.0754836	total: 44.3s	remaining: 7.21s
344:	learn: 0.0754601	total: 44.5s	remaining: 7.09s
345:	learn: 0.0754495	total: 44.6s	remaining: 6.97s
346:	learn: 0.0753936	total: 44.8s	remaining: 6.85s
347:	learn: 0.0753301	total: 45.1s	remaining: 6.73s
348:	learn: 0.0752830	total: 45.3s	remaining: 6.61s
349:	learn: 0.0752610	total: 45.4s	remaining: 6.49s
350:	learn: 0.075

In [30]:

Stack_final_X_train = np.concatenate(( cat_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate(( cat_test, lgbm_test), axis=1)
print('원본 학습 피처 데이터 Shape:',x_train.shape, '원본 테스트 피처 Shape:',test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape: (472432, 188) 원본 테스트 피처 Shape: (506691, 188)
스태킹 학습 피처 데이터 Shape: (472432, 2) 스태킹 테스트 피처 데이터 Shape: (506691, 2)


In [32]:
from sklearn.linear_model import LogisticRegression
lr_final = LogisticRegression()
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

In [33]:
stack_final

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
submission = pd.read_csv('/Users/krc/IEEE-CIS_Fraud_Detection/sample_submission.csv')

In [35]:
submission['isFraud']=stack_final

In [36]:
submission.head(3)

,TransactionID,isFraud
0,3663549,0
1,3663550,0
2,3663551,0


In [37]:
submission.to_csv('year_dream3.csv',index=False)